In [1]:
# !wget https://github.com/bamps53/convnext-tf/releases/download/v0.1/convnext_base_22k_1k_224.h5

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import tensorflow as tf
import numpy as np
from convnext import ConvNeXt, model_configs

In [4]:
x = tf.keras.Input(shape=(None, None, 3))
x_placeholder = tf.placeholder(tf.float32, shape = (None, None, None, 3))

In [5]:
tf.keras.layers.Input(tensor=x_placeholder)

<tf.Tensor 'Placeholder:0' shape=(?, ?, ?, 3) dtype=float32>

In [6]:
def preprocess_input(x):
    x = x.astype(np.float32)
    x /= 255.
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    x[..., 0] -= mean[0]
    x[..., 1] -= mean[1]
    x[..., 2] -= mean[2]
    x[..., 0] /= std[0]
    x[..., 1] /= std[1]
    x[..., 2] /= std[2]
    return x

In [7]:
from skimage.io import imread
from skimage.transform import resize

In [8]:
image = imread('panda.jpg')
image.shape

(1350, 2025, 3)

In [9]:
image_resized = resize(image, (224, 224))
image_resized = preprocess_input(image_resized)
image_resized = np.expand_dims(image_resized, 0)
image_resized.shape

(1, 224, 224, 3)

In [10]:
num_classes=1000
include_top=True
cfg = model_configs['convnext_base']
net = ConvNeXt(num_classes, cfg['depths'],
                   cfg['dims'], include_top)

In [11]:
out = net(x_placeholder)
out

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'conv_ne_xt/dense_72/BiasAdd:0' shape=(?, 1000) dtype=float32>

In [12]:
backbone = tf.keras.Model(x_placeholder, out)
backbone

In [13]:
net.load_weights('convnext_base_22k_1k_224.h5')

In [14]:
session = tf.keras.backend.get_session()

In [15]:
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [16]:
y = session.run(backbone.output, feed_dict = {x_placeholder: image_resized})

In [17]:
decode_predictions(y)

[[('n02510455', 'giant_panda', 6.7408805),
  ('n02509815', 'lesser_panda', 1.8700832),
  ('n04254680', 'soccer_ball', 1.6291409),
  ('n02134084', 'ice_bear', 1.4227448),
  ('n02134418', 'sloth_bear', 1.1176785)]]

In [18]:
saver = tf.train.Saver()
saver.save(session, "convnext_base_22k_1k_224/model.ckpt")

'convnext_base_22k_1k_224/model.ckpt'